## nltk_util.py

In [1]:
import nltk
from nltk.stem.porter import PorterStemmer
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
nltk.download('punkt')

In [38]:
def stem(word):
    stemmer = PorterStemmer()
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag



def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [39]:
example = "how are you today?"
print(example)
print(tokenize(example))

In [40]:
example = ["organize", "organizing", "organizes"]
print(example)
stemmed_words = [stem(w) for w in example]
print(stemmed_words)

## train.py

In [7]:
with open('../input/intents/intents.json', 'r') as f:
    intents = json.load(f)

    
# print(intents['intents'][0]['responses'])
all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))
ignore_words = ['?', '!', '.', ',']
all_words = [stem(w) for w in all_words if w not in ignore_words]

all_words = sorted(set(all_words))
tags = sorted(set(tags))


In [8]:
X_train = []
y_train = []
for (pattern, tag) in xy:
    bag = bag_of_words(pattern, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [9]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train
        
    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]
    
    def __len__(self):
        return self.n_samples
dataset = ChatDataset()

In [10]:
batch_size = 8

In [11]:
train_loader = DataLoader(dataset= dataset, batch_size = batch_size, shuffle = True, num_workers = 2)

## model.py

In [17]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.L1 = nn.Linear(input_size, hidden_size)
        self.L2 = nn.Linear(hidden_size, hidden_size)
        self.L3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    def forward(self, x):
        out = self.L1(x)
        out= self.relu(out)
        out = self.L2(out)
        out= self.relu(out)
        out = self.L3(out)
        
        return out

In [18]:
input_size  = len(X_train[0])
hidden_size = 8
num_classes = len(tags)

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size=input_size, hidden_size=hidden_size, num_classes=num_classes).to(device)

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [30]:
for epoch in range(1000):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)
        
        #forward
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        #backward and optim step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1)%100 == 0:
        print(f'epoch {epoch+1}/{1000}, loss={loss.item():.4f}')

print(f'final loss = {loss.item():.4f}')

In [31]:
data = {
    "model_state" : model.state_dict(),
    "input_size" : input_size,
    "output_size": num_classes,
    "hidden_size" : hidden_size,
    "all_words" : all_words,
    "tags" : tags
    
}

File = "data.pth"
torch.save(data, File)

print('training completed and file saved to {File}')

In [34]:
import random
import json

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
with open('../input/intents/intents.json', 'r') as f:
    intents = json.load(f)

file= 'data.pth'
torch.load(file)

input_size = data['input_size']
output_size = data['output_size']
hidden_size = data['hidden_size']
all_words = data['all_words']
tags = data['tags']
model_state = data['model_state']

model = NeuralNet(input_size=input_size, hidden_size=hidden_size, num_classes=output_size).to(device)
model.load_state_dict(model_state)
model.eval()

In [37]:
bot_name = "Grover"
print("Let's chat! type 'quit' to exit: ")

while True:
    sentence = input('You:' )
    if(sentence == 'quit'):
        break
    
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X)
    
    output = model(X)
    _, predicted = torch.max(output, dim=1)
    tag = tags[predicted.item()]
    
    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    
    if(prob.item() > 0.75):
        for intent in intents["intents"]:
            if (tag == intent["tag"]):
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand what you are saying sorry!")